<a href="https://colab.research.google.com/github/mscholl96/mad-recime/blob/network_LSTM/network/LSTM/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

## Basic includes

In [61]:
!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.73 s (started: 2022-03-14 18:56:23 +00:00)


In [62]:
!pip install word2vec
!pip install ray[tune]

time: 5.66 s (started: 2022-03-14 18:56:26 +00:00)


In [63]:
import word2vec
from collections import Counter # https://pymotw.com/2/collections/counter.html

import pandas as pd
import numpy as np
import itertools
import re
import os

import glob

time: 3.25 ms (started: 2022-03-14 18:56:31 +00:00)


In [64]:
from google.colab import drive
drive.mount('/content/drive/')

dataPath = '/content/drive/MyDrive/TP2/Datasets/Recipe1M/'
import sys
sys.path.append(dataPath)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
time: 1.99 s (started: 2022-03-14 18:56:31 +00:00)


## Import Data

In [65]:
TIMESTAMP = '2022_02_11'

time: 966 µs (started: 2022-03-14 18:56:33 +00:00)


In [66]:
baseFrame = pd.DataFrame()

smallSet = True

if(os.path.exists(dataPath + TIMESTAMP + '/recipes_valid_full.pkl')):
  baseFrame = pd.read_pickle(dataPath + TIMESTAMP + '/recipes_valid_full.pkl')
elif(smallSet == True):
  baseFrame = baseFrame.append(pd.read_pickle(glob.glob(dataPath + TIMESTAMP +  '/recipes_valid_*.pkl')[0]))
elif(len(glob.glob(dataPath + TIMESTAMP +  '/recipes_valid_*.pkl')) != 0):
  for file in glob.glob(dataPath + TIMESTAMP +  '/recipes_valid_*.pkl'):
    if not 'full' in file:
      baseFrame = baseFrame.append(pd.read_pickle(file))

baseFrame.head()

,title,ingredients,instructions
id,,,
000033e39b,Dilly Macaroni Salad Recipe,amount unit ingredient 0 1....,0 Cook macaroni according to package direct...
000035f7ed,Gazpacho,amount unit ingredient 0 8.0 ...,0 Add the tomatoes to a food processor with...
00003a70b1,Crunchy Onion Potato Bake,amount unit ingredient 0 2...,0 Preheat oven to 350 degrees Fah...
00004320bb,Cool 'n Easy Creamy Watermelon Pie,amount unit ingredient 0 3....,0 Dissolve Jello in boiling water. 1 ...
0000631d90,Easy Tropical Beef Skillet,amount unit ingredient 0...,"0 In a large skillet, toast the coconut ove..."


time: 28.8 s (started: 2022-03-14 18:56:33 +00:00)


In [67]:
# todo: move to dataset class
def flatten_ing(row):
  row['amount'] = row['ingredients']['amount'].tolist()
  row['unit'] = row['ingredients']['unit'].tolist()
  row['ingredient'] = row['ingredients']['ingredient'].tolist()
  return row

baseFrame = baseFrame.apply(flatten_ing, axis=1)
baseFrame = baseFrame.drop(columns=['ingredients'])

baseFrame.head()

,title,instructions,amount,unit,ingredient
id,,,,,
000033e39b,Dilly Macaroni Salad Recipe,0 Cook macaroni according to package direct...,"[1.0, 1.0, 0.5, 0.5, 3.0, 0.5, 1.0, 0.75, 0.5]","[, , , , tablespoon, , tablespoon, teaspoon, t...","[elbow macaroni, american cheese, celery, gree..."
000035f7ed,Gazpacho,0 Add the tomatoes to a food processor with...,"[8.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 3.0]","[, , , , , , , , ]","[tomatoes, kosher salt, red onion, green bell ..."
00003a70b1,Crunchy Onion Potato Bake,0 Preheat oven to 350 degrees Fah...,"[2.5, 1.5, 0.25, 1.0, 8.0, 1.0, 1.0]","[, , cup, , ounce, cup, cup]","[milk, water, butter, mashed potatoes, whole k..."
00004320bb,Cool 'n Easy Creamy Watermelon Pie,0 Dissolve Jello in boiling water. 1 ...,"[3.0, 0.25, 12.0, 2.0, 1.0]","[ounce, cup, ounce, cup, ]","[watermelon gelatin, boiling water, cool whip,..."
0000631d90,Easy Tropical Beef Skillet,"0 In a large skillet, toast the coconut ove...","[0.5, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 8.0, 16.0,...","[cup, pound-mass, tablespoon, , tablespoon, ta...","[shredded coconut, lean ground beef, fresh gar..."


time: 2min 57s (started: 2022-03-14 18:57:02 +00:00)


## Brainstorming
* not a simple sequence
* basically sequence of
  * ingredients
  * set of instructions --> instruction --> words
* maybe instead of instructions sequences, instructions could be flattened and split signs (.,!?;) also learned (in case learning sentence sequences is not possible)

Maybe stacked LSTMs are a solutions
* train how instructions in general are to be composed
* train how instructions are combined to a set of instructions

Input
* ingredients to be used as input
* to not train on instructions to appear according to ingredients sequence, maybe randomly shuffle ingredients but keep instruction set the same
* 

https://www.youtube.com/watch?v=A9QVYOBjZdY&list=PLQY2H8rRoyvzDbLUZkbudP-MFQZwNmU4S&index=5&ab_channel=TensorFlow


## Differentiation
* Title generation based on ingredients
* instruction generation

## Imports for Learning
https://pytorch.org/tutorials/beginner/introyt/trainingyt.html

In [68]:
import torch

# Model
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable 
from torchsummary import summary

# Optimizer
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

# Tokenizer
# torch padding does only support constant padding (ConstantPad1d) for 1D or non-constant padding for >1D (nn.function.pad)
from tensorflow.keras.preprocessing.sequence import pad_sequences
# keras tokenizer more powerful than torch
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from torchtext.data import get_tokenizer # https://pytorch.org/text/stable/data_utils.html

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

# hyperparameter tuning
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

time: 7.24 ms (started: 2022-03-14 19:00:00 +00:00)


# Seed

In [69]:
torch.manual_seed(1)

time: 2.94 ms (started: 2022-03-14 19:00:00 +00:00)


# Setup
https://closeheat.com/blog/pytorch-lstm-text-generation-tutorial


## Tokenization
to be checked: necessity of punctuation (maybe reintroduce later: https://stackoverflow.com/questions/49073673/include-punctuation-in-keras-tokenizer)

### Get Corpus

In [70]:
w2v_model = word2vec.load(dataPath + 'vocab.bin')
ingredientDict = {}
for voc in w2v_model.vocab:
     # Offset by 1 so empty fields can be 0
     ingredientDict.setdefault(voc, len(ingredientDict)+1)

if 'dilly' in ingredientDict:
  print("Word exists")
else:
  print('vocab.bin not to be used as dict misses words') 

vocab.bin not to be used as dict misses words
time: 642 ms (started: 2022-03-14 19:00:00 +00:00)


In [71]:
def getCorpus(title, ingredient, instructions):
  titleTok = text_to_word_sequence(title)
  ingTok = text_to_word_sequence(' '.join(ingredient))
  instTok = text_to_word_sequence(' '.join(instructions))
  return np.array(ingTok + titleTok + instTok)

corpus = np.vectorize(getCorpus, otypes=[np.ndarray])(baseFrame['title'], baseFrame['ingredient'], baseFrame['instructions'])
corpus = np.concatenate(corpus)

time: 7.14 s (started: 2022-03-14 19:00:01 +00:00)


###  Sequences

In [72]:
test = baseFrame.head(20).copy()

def getTitleSequence(title, ingredient):
  titleTok = text_to_word_sequence(title)
  ingTok = text_to_word_sequence(','.join(ingredient))
  return {'ings': ingTok, 'title': titleTok}

titleSeq = np.vectorize(getTitleSequence, otypes=[np.ndarray])(test['title'], test['ingredient'])
titleSeq[0]

{'ings': ['elbow',
  'macaroni',
  'american',
  'cheese',
  'celery',
  'green',
  'peppers',
  'pimentos',
  'mayonnaise',
  'vinegar',
  'salt',
  'dry',
  'dill',
  'weed'],
 'title': ['dilly', 'macaroni', 'salad', 'recipe']}

time: 5.89 ms (started: 2022-03-14 19:00:08 +00:00)


In [73]:
def getNGramSeq(seq):
  # input needs to be pre padded
  idxShift = len(seq['title'])
  ingLen = len(seq['ings'])
  fullSeq = np.array(seq['ings'] + seq['title'])
  retSeq = np.empty((0,ingLen + 1))
  for i_shift in range(idxShift):
    retSeq = np.vstack([retSeq, np.array(fullSeq[i_shift:ingLen+i_shift+1])])
  return retSeq

ngramSeq = pd.Series(np.vectorize(getNGramSeq, otypes=[np.ndarray])(titleSeq)).explode().to_numpy()
ngramSeq[0][:-1], ngramSeq[0][1:]

(array(['elbow', 'macaroni', 'american', 'cheese', 'celery', 'green',
        'peppers', 'pimentos', 'mayonnaise', 'vinegar', 'salt', 'dry',
        'dill', 'weed'], dtype='<U32'),
 array(['macaroni', 'american', 'cheese', 'celery', 'green', 'peppers',
        'pimentos', 'mayonnaise', 'vinegar', 'salt', 'dry', 'dill', 'weed',
        'dilly'], dtype='<U32'))

time: 9.67 ms (started: 2022-03-14 19:00:08 +00:00)


### Hyperparams

In [105]:
class HyperParams():
  def __init__(self, epochs=10, batchSize=10, lr=1e-3, trainValRatio=[0.7, 0.3]):
    self.epochs = range(1, epochs+1)
    self.batchSize = batchSize
    self.lr = lr
    self.trainValRatio = trainValRatio

    self.input_size = 5 #number of features
    self.hidden_dim = 4 #number of features in hidden state
    self.num_layers = 1 #number of stacked lstm layers
    self.num_classes = 1 #number of output classes 
    self.embedding_dim = 4 # embedding dimension

  def __str__(self):
    return('epochs ' + str(self.epochs) + '\n' +
    'batchSize ' + str(self.batchSize) + '\n' +
    'lr ' + str(self.lr) + '\n' +
    'trainValRatio ' + str(self.trainValRatio) + '\n' +
    'input_size ' + str(self.input_size) + '\n' +
    'hidden_dim ' + str(self.hidden_dim) + '\n' +
    'num_layers ' + str(self.num_layers) + '\n' +
    'num_classes ' + str(self.num_classes) + '\n' +
    'embedding_dim ' + str(self.embedding_dim) + '\n')
    

time: 8.76 ms (started: 2022-03-14 20:11:52 +00:00)


### Dataset

In [208]:
# Data (https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class)
class TitleDataset(Dataset):
    def __init__(self, hyperparams, data):
      self.hyperparams = hyperparams

      self.tokenizer = Tokenizer()

      # dataset split into word sequences required for training
      self.wordSeq = np.vectorize(self.getTitleSequence, otypes=[np.ndarray])(data['title'], data['ingredient'])

      # training requires same length sequences -->  padding
      self.maxSequenceLength = max([max(len(seq['ings']), len(seq['title'])) for seq in self.wordSeq])

      # list of all words in dataset
      self.words = np.concatenate(np.vectorize(self.getCorpus, otypes=[np.ndarray])(data['title'], data['ingredient'], data['instructions']))

      # tokenization corpus
      self.tokenizer.fit_on_texts(self.words)

      # indexed wordSequences (could be calculated in getter but very slow, preprocessing better)
      self.idxWords = np.vectorize(self.getIndexedSeqs, otypes=[np.ndarray])(self.wordSeq)

      # n gram sequences
      self.ngramSeq = pd.Series(np.vectorize(self.getNGramSeq, otypes=[np.ndarray])(self.idxWords)).explode().to_numpy()


    def getCorpus(self, title, ingredient, instructions):
      titleTok = text_to_word_sequence(title)
      ingTok = text_to_word_sequence(','.join(ingredient))
      # instTok = text_to_word_sequence('\n'.join(instructions)) # TODO: remove
      return np.array(ingTok + titleTok)# + instTok)

    def getTitleSequence(self, title, ingredient):
      titleTok = text_to_word_sequence(title)
      ingTok = text_to_word_sequence(','.join(ingredient))
      return {'ings': ingTok, 'title': titleTok}

    def getIndexedSeqs(self, seq):
      ingTok = self.tokenizer.texts_to_sequences([seq['ings']])[0]
      ingTok = pad_sequences([ingTok], maxlen=self.maxSequenceLength, padding='pre')[0] # https://arxiv.org/abs/1903.07288
      titleTok = self.tokenizer.texts_to_sequences([seq['title']])[0]
      # titleTok = pad_sequences([titleTok], maxlen=self.maxSequenceLength, padding='post')[0] # post in order to get oov to indicate end of title

      return {'ings': ingTok, 'title': titleTok}

    def getNGramSeq(self, seq):
      # input needs to be pre padded
      idxShift = len(seq['title'])
      ingLen = len(seq['ings'])

      fullSeq = np.append(seq['ings'], seq['title'])
      retSeq = np.empty((0,ingLen + 1), dtype=np.int32)

      for i_shift in range(idxShift):
        retSeq = np.vstack([retSeq, np.array(fullSeq[i_shift:ingLen+i_shift+1])])
      return retSeq

    def __len__(self):
        return len(self.idxWords)

    def __getitem__(self, index):
      # tuple of input (ingredients) and label (title)
        return (
            torch.tensor(self.ngramSeq[index][:-1]),
            torch.tensor(self.ngramSeq[index][1:])
        )

time: 62.8 ms (started: 2022-03-14 21:00:55 +00:00)


## Model
LSTM Net: https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html

Embedding Net: https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html

Init state: https://stats.stackexchange.com/questions/224737/best-way-to-initialize-lstm-state

### base: https://cnvrg.io/pytorch-lstm/
Without embedding layer before --> todo: check how this is better/worse

### base: https://github.com/yuchenlin/lstm_sentence_classifier/blob/master/LSTM_sentence_classifier.py

### base: https://www.analyticsvidhya.com/blog/2020/08/build-a-natural-language-generation-nlg-system-using-pytorch/

### base: https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#lstms-in-pytorch

In [205]:
class Model3(nn.Module):

    def __init__(self, hyperParams, dataset, device):
        super(Model3, self).__init__()

        # initialize vital params
        self.vocab_size = len(dataset.tokenizer.word_index)
        self.batchSize = hyperParams.batchSize
        self.hidden_dim = hyperParams.hidden_dim
        self.device = device
        self.num_layers = hyperParams.num_layers
        
        # embedding definition 
        self.word_embeddings = nn.Embedding(self.vocab_size, hyperParams.embedding_dim)

        # lstm definition
        self.lstm = nn.LSTM(input_size=hyperParams.embedding_dim, hidden_size=hyperParams.hidden_dim, num_layers=self.num_layers)

        # definition fully connected layer
        self.hidden2label = nn.Linear(hyperParams.hidden_dim, self.vocab_size)

    def forward(self, x, hidden):
        embeds = self.word_embeddings(x)
        lstm_out, hidden = self.lstm(embeds.view(len(x[0]), self.batchSize, -1), hidden)
        out = self.hidden2label(lstm_out[-1])
        # tag_scores = F.log_softmax(tag_space, dim=1)
        return out, hidden

    def init_hidden(self):
        ''' initializes hidden state '''
        # Create two new tensors with sizes num_layers x batchSize x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        hidden = (weight.new(self.num_layers, self.batchSize, self.hidden_dim).zero_(),
                  weight.new(self.num_layers, self.batchSize, self.hidden_dim).zero_())
        
        return hidden

time: 18.2 ms (started: 2022-03-14 20:59:38 +00:00)


## Training
mixture of 
* https://pytorch.org/tutorials/beginner/introyt/trainingyt.html
* https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html
* https://stackoverflow.com/questions/67295494/correct-validation-loss-in-pytorch


In [217]:
def train(dataset, model, hyperparams, device):
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=hyperparams.lr)

  model = model.to(device)

  # split data
  train_set, val_set = random_split(dataset, [int(hyperparams.trainValRatio[0] * len(dataset)), int(hyperparams.trainValRatio[1] * len(dataset))], generator=torch.Generator().manual_seed(42))

  train_loader = DataLoader(train_set, batch_size=hyperparams.batchSize)
  val_loader   = DataLoader(val_set, batch_size=hyperparams.batchSize)
  # further options: shuffle, num_workers

  for epoch in hyperparams.epochs:
    running_loss = 0.
    epoch_steps = 0

    h = model.init_hidden()

    # model.train() # what does it do

    for batch, (inputs, labels) in enumerate(train_loader):
      # assign inputs and labels to device
      inputs, labels = inputs.to(device), labels.to(device)

      # detach hidden states
      h = tuple([each.data for each in h])

      # clear gradients
      optimizer.zero_grad()

      # batch prediction (alternative: forward)
      print('Sentence: ' + str(inputs.shape))
      outputs, h = model(inputs, h)
      # outputs = outputs.to(device)
      # h = h.to(device)

      # loss computation
      print('Outputs: ' + str(outputs.shape))
      print(outputs)
      print('Labels: ' + str(labels.shape))

      # loss = criterion(torch.argmax(outputs, dim=1, keepdim=True), labels)
      loss = criterion(outputs, labels)

      # calc backward gradients
      loss.backward()

      # run optimizer
      optimizer.step()

      # print statistics
      running_loss += loss.item()
      epoch_steps += 1

      if epoch % 5 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, running_loss / epoch_steps)) 
        running_loss = 0.0

      
    # Validation Loss
    correct = 0                                               
    total = 0                                                 
    running_loss = 0.0    
    val_steps = 0                                    
    
    model.eval() # what does it do
    with torch.no_grad(): # what does it do
      for batch, (inputs, labels) in enumerate(val_loader):
        # assign inputs and labels to device
        # input, labels = inputs.to(device), labels.to(device)

        # batch prediction (alternative: forward)
        outputs = model(inputs, labels)

        outputs = outputs.to(device)

        # loss computation
        loss = criterion(outputs, labels)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()
        val_steps += 1

    mean_val_accuracy = (100 * correct / total)               
    mean_val_loss = ( running_loss )                  
    print('Validation Accuracy: %d %%' % (mean_val_accuracy)) 
    print('Validation Loss:'  ,mean_val_loss )


time: 115 ms (started: 2022-03-14 21:12:11 +00:00)


In [193]:
def predict(dataset, model, words, next_words=100):
    model.eval()

    state_h, state_c = model.init_state(len(words))
    title = []

    for i in range(0, next_words):
        x = torch.tensor([[dataset.tokenizer.word_index[w] for w in words[i:]]])
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))

        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(dataset.tokenizer.sequences_to_texts([[word_index]])[0])
        title.append(dataset.tokenizer.sequences_to_texts([[word_index]])[0])

    return title

time: 10.6 ms (started: 2022-03-14 20:56:54 +00:00)


# USE TENSORBOARXD (looks fancy, good for analysis)

## Execution

In [194]:
hyperParams = HyperParams()
print(hyperParams)

epochs range(1, 11)
batchSize 10
lr 0.001
trainValRatio [0.7, 0.3]
input_size 5
hidden_dim 4
num_layers 1
num_classes 1
embedding_dim 4

time: 940 µs (started: 2022-03-14 20:56:56 +00:00)


In [209]:
titleSet = TitleDataset(hyperParams, baseFrame)

time: 23.7 s (started: 2022-03-14 21:01:04 +00:00)


In [210]:
titleSet.tokenizer.word_index

{'salt': 1,
 'pepper': 2,
 'sugar': 3,
 'oil': 4,
 'butter': 5,
 'fresh': 6,
 'cheese': 7,
 'garlic': 8,
 'ground': 9,
 'flour': 10,
 'chicken': 11,
 'cream': 12,
 'and': 13,
 'sauce': 14,
 'powder': 15,
 'olive': 16,
 'water': 17,
 'black': 18,
 'milk': 19,
 'red': 20,
 'juice': 21,
 'onions': 22,
 'onion': 23,
 'lemon': 24,
 'eggs': 25,
 'vanilla': 26,
 'white': 27,
 'baking': 28,
 'cloves': 29,
 'egg': 30,
 'green': 31,
 'with': 32,
 'chocolate': 33,
 'tomatoes': 34,
 'dried': 35,
 'brown': 36,
 'vinegar': 37,
 'all': 38,
 'cinnamon': 39,
 'purpose': 40,
 'vegetable': 41,
 'parsley': 42,
 'beef': 43,
 'extract': 44,
 'wine': 45,
 'rice': 46,
 'leaves': 47,
 'potatoes': 48,
 'peppers': 49,
 'bread': 50,
 'salad': 51,
 'sweet': 52,
 'orange': 53,
 'mustard': 54,
 'beans': 55,
 'soup': 56,
 'chili': 57,
 'soda': 58,
 'parmesan': 59,
 'unsalted': 60,
 'tomato': 61,
 'whole': 62,
 'corn': 63,
 'ginger': 64,
 'dry': 65,
 'sour': 66,
 'celery': 67,
 'fat': 68,
 'broth': 69,
 'basil': 70,
 

time: 17.2 ms (started: 2022-03-14 21:01:27 +00:00)


In [211]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(0))

Tesla T4
time: 1.18 ms (started: 2022-03-14 21:01:28 +00:00)


In [212]:
model = Model3(hyperParams, titleSet, device)
print(model)
# summary(model, (16,53))

Model3(
  (word_embeddings): Embedding(19623, 4)
  (lstm): LSTM(4, 4)
  (hidden2label): Linear(in_features=4, out_features=19623, bias=True)
)
time: 3.81 ms (started: 2022-03-14 21:01:28 +00:00)


In [213]:
train_loader = DataLoader(titleSet, batch_size=hyperParams.batchSize)
for batch, (inputs, labels) in enumerate(train_loader):
  print(len(inputs[0]))
  test = model.word_embeddings(inputs)
  print(test.shape)
  lstm_out, _ = model.lstm(test.view(len(inputs[0]), model.batchSize, -1))
  print(lstm_out.shape)
  break;

53
torch.Size([10, 53, 4])
torch.Size([53, 10, 4])
time: 9.36 ms (started: 2022-03-14 21:01:28 +00:00)


In [218]:
train(titleSet, model, hyperParams, device)

Sentence: torch.Size([10, 53])
Outputs: torch.Size([10, 19623])
tensor([[-0.0656, -0.3455,  0.0586,  ...,  0.1060, -0.0769,  0.1026],
        [-0.0343, -0.1290, -0.0653,  ..., -0.0211, -0.2027, -0.0505],
        [ 0.0324, -0.0539,  0.0165,  ..., -0.1404, -0.1946, -0.1294],
        ...,
        [ 0.0145, -0.1767, -0.0038,  ..., -0.0453, -0.1849, -0.0017],
        [ 0.0305, -0.2493,  0.0719,  ..., -0.0435, -0.1117,  0.0213],
        [ 0.0163, -0.1927,  0.0315,  ..., -0.0628, -0.1251, -0.0409]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
Labels: torch.Size([10, 53])


RuntimeError: ignored

time: 49 ms (started: 2022-03-14 21:12:16 +00:00)


In [ ]:
idx = 100
print('Orig: ' + str(titleSet.wordSeq[idx]['title']) + '\n' + 
      'Predict: ' + str(predict(titleSet, model, words=titleSet.wordSeq[idx]['ings'], next_words=len(titleSet.wordSeq[idx]['title']))))

In [ ]:
a = torch.randn(4, 4)
a.shape
torch.argmax(a, dim=1, keepdim=True).shape

https://towardsdatascience.com/lstm-for-time-series-prediction-de8aeb26f2ca for input sequence